# BIOSTAT 826 - Assignment 1
Mortality prediction in MIMIC-IV using ICD-10 diagnosis and procedure categories.

## Setup

In [ ]:
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

repo_root = Path.cwd()
if not (repo_root / 'utils').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from utils.data import assemble_dataset, load_code_descriptions
from utils.evaluation import (
    TemperatureScaler,
    bootstrap_metric_ci,
    calibration_curve_quantile,
    calibration_slope_intercept,
    compute_basic_metrics,
    sigmoid,
    specificity_at_sensitivity,
)
from utils.training import (
    LogisticRegressionModel,
    MLPRiskModel,
    build_dataloader,
    coefficient_table,
    extract_linear_weights,
    predict_logits,
    set_seed,
    train_lr_sweep_with_models,
    train_model,
    tune_l1_strength,
)

In [ ]:
set_seed(826)
data_dir = Path('/home/rl/mimic-iv-3.1/mimic-iv-3.1/hosp')
bundle = assemble_dataset(data_dir, min_count=10, seed=826)
desc = load_code_descriptions(data_dir)

X = bundle.X
y = bundle.y.astype(np.float32)
splits = bundle.splits
feature_names = bundle.feature_names

X_train, y_train = X[splits['train']], y[splits['train']]
X_val, y_val = X[splits['val']], y[splits['val']]
X_cal, y_cal = X[splits['cal']], y[splits['cal']]
X_test, y_test = X[splits['test']], y[splits['test']]

print('n_admissions:', X.shape[0])
print('n_features:', X.shape[1])
print('mortality_rate:', float(y.mean()))

### Feature scaling for MLP
Use train-set standardization (no centering for sparse matrix). We keep raw binary features for LR interpretability, and use scaled features for MLP fitting stability.

In [ ]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_cal_scaled = scaler.transform(X_cal)
X_test_scaled = scaler.transform(X_test)

## Part 2 - Logistic Regression

### LR sweep means trying multiple learning rates
Here we train the same logistic regression with different learning rates and compare optimization behavior.

In [ ]:
train_loader = build_dataloader(X_train, y_train, batch_size=200, shuffle=True)
val_loader = build_dataloader(X_val, y_val, batch_size=200, shuffle=False)
cal_loader = build_dataloader(X_cal, y_cal, batch_size=200, shuffle=False)
test_loader = build_dataloader(X_test, y_test, batch_size=200, shuffle=False)

In [ ]:
lrs = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
lr_sweep = train_lr_sweep_with_models(
    lambda: LogisticRegressionModel(X.shape[1]),
    train_loader,
    val_loader,
    lrs,
    max_epochs=40,
    patience=6,
)

lr_summary = []
for lr, trained in lr_sweep.items():
    lr_summary.append({
        'lr': lr,
        'epochs': len(trained.result.history['epoch_train_loss']),
        'best_epoch': trained.result.best_epoch,
        'best_val_loss': min(trained.result.history['val_loss']),
    })
lr_summary_df = pd.DataFrame(lr_summary).sort_values('lr').reset_index(drop=True)
lr_summary_df

In [ ]:
plt.figure(figsize=(9, 5))
for lr, trained in lr_sweep.items():
    plt.plot(trained.result.history['batch_train_loss'], label=f'lr={lr:g}', alpha=0.9)
plt.xlabel('Mini-batch step')
plt.ylabel('Mini-batch BCE loss')
plt.title('Logistic regression mini-batch loss by learning rate')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
best_lr = float(lr_summary_df.sort_values('best_val_loss').iloc[0]['lr'])
lr_best_model = lr_sweep[best_lr].model
lr_best_weights = extract_linear_weights(lr_best_model)
lr_top20, lr_bottom20 = coefficient_table(lr_best_weights, feature_names, desc, top_k=20)

print(f'Best LR by validation loss: {best_lr:g}')
print('Top 20 categories increasing log-odds')
display(lr_top20)
print('Top 20 categories decreasing log-odds')
display(lr_bottom20)

In [ ]:
l1_grid = [0.0, 1e-6, 3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]
chosen_l1, l1_trained, l1_table = tune_l1_strength(
    lambda: LogisticRegressionModel(X.shape[1]),
    train_loader,
    val_loader,
    l1_grid,
    target_sparsity=0.85,
    lr=best_lr,
    max_epochs=40,
    patience=6,
)

print('Chosen L1 lambda:', chosen_l1)
l1_table

In [ ]:
l1_weights = extract_linear_weights(l1_trained.model)
l1_top20, l1_bottom20 = coefficient_table(l1_weights, feature_names, desc, top_k=20)

print('L1 model top 20 categories increasing log-odds')
display(l1_top20)
print('L1 model top 20 categories decreasing log-odds')
display(l1_bottom20)

## Part 3 - Neural Network Risk Predictor

In [ ]:
def run_nn_experiments(X_train_local, y_train_local, X_val_local, y_val_local, hidden_sizes):
    train_local = build_dataloader(X_train_local, y_train_local, batch_size=200, shuffle=True)
    val_local = build_dataloader(X_val_local, y_val_local, batch_size=200, shuffle=False)
    results = {}
    input_dim = X_train_local.shape[1]
    for hidden_dim in hidden_sizes:
        model = MLPRiskModel(input_dim, hidden_dim=hidden_dim)
        result = train_model(
            model,
            train_local,
            val_local,
            lr=1e-3,
            max_epochs=35,
            patience=6,
        )
        results[hidden_dim] = {'model': model, 'result': result}
    return results

In [ ]:
hidden_sizes = [100, 1000, 5000]
nn_full = run_nn_experiments(X_train_scaled, y_train, X_val_scaled, y_val, hidden_sizes)

nn_full_summary = []
for h, item in nn_full.items():
    nn_full_summary.append({
        'hidden_size': h,
        'epochs': len(item['result'].history['epoch_train_loss']),
        'best_epoch': item['result'].best_epoch,
        'best_val_loss': min(item['result'].history['val_loss']),
    })
nn_full_summary_df = pd.DataFrame(nn_full_summary).sort_values('hidden_size').reset_index(drop=True)
nn_full_summary_df

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4), sharey=True)
for i, h in enumerate(hidden_sizes):
    hist = nn_full[h]['result'].history
    axes[i].plot(hist['epoch_train_loss'], label='train')
    axes[i].plot(hist['val_loss'], label='val')
    axes[i].set_title(f'Hidden={h}')
    axes[i].set_xlabel('Epoch')
    if i == 0:
        axes[i].set_ylabel('Loss')
        axes[i].legend()
plt.tight_layout()
plt.show()

In [ ]:
rng = np.random.default_rng(826)
downsample_n = max(1, len(y_train) // 10)
down_idx = rng.choice(np.arange(len(y_train)), size=downsample_n, replace=False)
X_train_down = X_train_scaled[down_idx]
y_train_down = y_train[down_idx]

nn_down = run_nn_experiments(X_train_down, y_train_down, X_val_scaled, y_val, hidden_sizes)

nn_down_summary = []
for h, item in nn_down.items():
    nn_down_summary.append({
        'hidden_size': h,
        'epochs': len(item['result'].history['epoch_train_loss']),
        'best_epoch': item['result'].best_epoch,
        'best_val_loss': min(item['result'].history['val_loss']),
    })
nn_down_summary_df = pd.DataFrame(nn_down_summary).sort_values('hidden_size').reset_index(drop=True)
nn_down_summary_df

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4), sharey=True)
for i, h in enumerate(hidden_sizes):
    hist = nn_down[h]['result'].history
    axes[i].plot(hist['epoch_train_loss'], label='train')
    axes[i].plot(hist['val_loss'], label='val')
    axes[i].set_title(f'Downsampled Hidden={h}')
    axes[i].set_xlabel('Epoch')
    if i == 0:
        axes[i].set_ylabel('Loss')
        axes[i].legend()
plt.tight_layout()
plt.show()

## Part 4 - Performance Evaluation

In [ ]:
best_nn_hidden = int(nn_full_summary_df.sort_values('best_val_loss').iloc[0]['hidden_size'])
nn_best_model = nn_full[best_nn_hidden]['model']

lr_logits_cal = predict_logits(lr_best_model, cal_loader)
lr_logits_test = predict_logits(lr_best_model, test_loader)
nn_cal_loader = build_dataloader(X_cal_scaled, y_cal, batch_size=200, shuffle=False)
nn_test_loader = build_dataloader(X_test_scaled, y_test, batch_size=200, shuffle=False)
nn_logits_cal = predict_logits(nn_best_model, nn_cal_loader)
nn_logits_test = predict_logits(nn_best_model, nn_test_loader)

lr_scaler = TemperatureScaler()
nn_scaler = TemperatureScaler()
lr_temp = lr_scaler.fit(lr_logits_cal, y_cal)
nn_temp = nn_scaler.fit(nn_logits_cal, y_cal)

lr_prob_raw = sigmoid(lr_logits_test)
lr_prob_cal = sigmoid(lr_logits_test / lr_temp)
nn_prob_raw = sigmoid(nn_logits_test)
nn_prob_cal = sigmoid(nn_logits_test / nn_temp)

print('Best NN hidden size:', best_nn_hidden)
print('Temperature LR:', lr_temp)
print('Temperature NN:', nn_temp)

In [ ]:
candidate_sens = [0.80, 0.85, 0.90]
sens_rows = []
for target in candidate_sens:
    lr_sens, lr_spec, _ = specificity_at_sensitivity(y_test, lr_prob_cal, target)
    nn_sens, nn_spec, _ = specificity_at_sensitivity(y_test, nn_prob_cal, target)
    sens_rows.append({
        'target_sensitivity': target,
        'lr_specificity': lr_spec,
        'nn_specificity': nn_spec,
        'mean_specificity': (lr_spec + nn_spec) / 2.0,
    })
sens_df = pd.DataFrame(sens_rows).sort_values('mean_specificity', ascending=False).reset_index(drop=True)
fixed_sensitivity = float(sens_df.iloc[0]['target_sensitivity'])
print('Selected fixed sensitivity:', fixed_sensitivity)
sens_df

In [ ]:
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_prob_cal)
fpr_nn, tpr_nn, _ = roc_curve(y_test, nn_prob_cal)

prec_lr, rec_lr, _ = precision_recall_curve(y_test, lr_prob_cal)
prec_nn, rec_nn, _ = precision_recall_curve(y_test, nn_prob_cal)

plt.figure(figsize=(7, 6))
plt.plot(fpr_lr, tpr_lr, label=f'LR (AUROC={roc_auc_score(y_test, lr_prob_cal):.3f})')
plt.plot(fpr_nn, tpr_nn, label=f'NN (AUROC={roc_auc_score(y_test, nn_prob_cal):.3f})')
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 6))
plt.plot(rec_lr, prec_lr, label=f'LR (AP={average_precision_score(y_test, lr_prob_cal):.3f})')
plt.plot(rec_nn, prec_nn, label=f'NN (AP={average_precision_score(y_test, nn_prob_cal):.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
lr_curve_raw = calibration_curve_quantile(y_test, lr_prob_raw, n_bins=15)
lr_curve_cal = calibration_curve_quantile(y_test, lr_prob_cal, n_bins=15)
nn_curve_raw = calibration_curve_quantile(y_test, nn_prob_raw, n_bins=15)
nn_curve_cal = calibration_curve_quantile(y_test, nn_prob_cal, n_bins=15)

plt.figure(figsize=(8, 7))
plt.plot([0, 1], [0, 1], '--', color='black', label='Perfect calibration')
plt.plot(lr_curve_raw['mean_pred'], lr_curve_raw['frac_pos'], 'o-', label='LR raw')
plt.plot(lr_curve_cal['mean_pred'], lr_curve_cal['frac_pos'], 'o-', label='LR temp-scaled')
plt.plot(nn_curve_raw['mean_pred'], nn_curve_raw['frac_pos'], 's-', label='NN raw')
plt.plot(nn_curve_cal['mean_pred'], nn_curve_cal['frac_pos'], 's-', label='NN temp-scaled')
plt.xlabel('Mean predicted probability (quantile bins)')
plt.ylabel('Observed event rate')
plt.title('Calibration Curves (Quantile Bins)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def metric_auroc(y_true, y_prob):
    return roc_auc_score(y_true, y_prob)

def metric_ap(y_true, y_prob):
    return average_precision_score(y_true, y_prob)

def metric_cal_slope(y_true, y_prob):
    slope, _ = calibration_slope_intercept(y_true, y_prob)
    return slope

def metric_cal_intercept(y_true, y_prob):
    _, intercept = calibration_slope_intercept(y_true, y_prob)
    return intercept

def metric_spec_at_fixed_sens(y_true, y_prob):
    _, spec, _ = specificity_at_sensitivity(y_true, y_prob, fixed_sensitivity)
    return spec

In [ ]:
def summarize_with_bootstrap(model_name, y_true, y_prob, n_bootstrap=1000):
    metrics = {
        'AUROC': metric_auroc,
        'AP': metric_ap,
        'Calibration slope': metric_cal_slope,
        'Calibration intercept': metric_cal_intercept,
        f'Specificity @ sensitivity={fixed_sensitivity:.2f}': metric_spec_at_fixed_sens,
    }
    rows = []
    for metric_name, fn in metrics.items():
        point, (lo, hi), _ = bootstrap_metric_ci(
            y_true,
            y_prob,
            fn,
            n_bootstrap=n_bootstrap,
            seed=826,
        )
        rows.append({
            'model': model_name,
            'metric': metric_name,
            'estimate': point,
            'ci_lower': lo,
            'ci_upper': hi,
        })
    return pd.DataFrame(rows)

result_lr = summarize_with_bootstrap('Logistic Regression', y_test, lr_prob_cal, n_bootstrap=1000)
result_nn = summarize_with_bootstrap('Neural Network', y_test, nn_prob_cal, n_bootstrap=1000)
results_df = pd.concat([result_lr, result_nn], ignore_index=True)
results_df